## Credit Scoring Model
Train and evaluate credit scoring models.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def generate_synthetic_credit_data(n=2000, random_state=42):
    rng = np.random.default_rng(random_state)
    age = rng.integers(21,75,n)
    income = np.clip(rng.normal(80000,30000,n),10000,300000)
    dti = np.clip(rng.beta(2,8,n)*1.2,0,1.5)
    credit_history_years = rng.integers(0,30,n)
    num_open_accounts = rng.integers(0,12,n)
    num_late = np.clip(rng.poisson(0.5,n),0,15)
    utilization = rng.beta(2.5,3.5,n)
    has_mortgage = rng.integers(0,2,n)
    existing_loans = rng.integers(0,5,n)
    payment_history = np.clip(rng.normal(0.75,0.15,n),0,1)
    z = 1.5*dti+1.2*utilization+0.5*num_late+0.3*existing_loans-0.8*payment_history-0.02*credit_history_years-0.000006*income
    prob = 1/(1+np.exp(-z))
    y = (rng.random(n)<prob).astype(int)
    return pd.DataFrame({'age':age,'income':income,'dti':dti,'credit_history_years':credit_history_years,
                         'num_open_accounts':num_open_accounts,'num_late_payments':num_late,'utilization':utilization,
                         'has_mortgage':has_mortgage,'existing_loans':existing_loans,'payment_history_score':payment_history,
                         'defaulted':y})

In [ ]:
df = generate_synthetic_credit_data()
df.head()

In [ ]:
X = df.drop(columns=['defaulted']); y = df['defaulted']
num_features=['age','income','dti','credit_history_years','num_open_accounts','num_late_payments','utilization','existing_loans','payment_history_score']
cat_features=['has_mortgage']
preprocessor = ColumnTransformer([('num',StandardScaler(),num_features),('cat',OneHotEncoder(drop='if_binary'),cat_features)])
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)

In [ ]:
models={'LogisticRegression':LogisticRegression(max_iter=200),
         'DecisionTree':DecisionTreeClassifier(max_depth=6,random_state=42),
         'RandomForest':RandomForestClassifier(n_estimators=200,random_state=42)}
fitted={};metrics={}
for name,clf in models.items():
    pipe=Pipeline([('prep',preprocessor),('clf',clf)])
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    y_proba=pipe.predict_proba(X_test)[:,1]
    metrics[name]={'precision':precision_score(y_test,y_pred),'recall':recall_score(y_test,y_pred),
                   'f1':f1_score(y_test,y_pred),'roc_auc':roc_auc_score(y_test,y_proba)}
    fitted[name]=pipe
metrics

In [ ]:
plt.figure()
for name,pipe in fitted.items():
    y_proba=pipe.predict_proba(X_test)[:,1]
    fpr,tpr,_=roc_curve(y_test,y_proba)
    plt.plot(fpr,tpr,label=f"{name} AUC={metrics[name]['roc_auc']:.3f}")
plt.plot([0,1],[0,1],'--');plt.xlabel('FPR');plt.ylabel('TPR');plt.legend();plt.title('ROC Curves');plt.show()

In [ ]:
best=max(metrics,key=lambda k:metrics[k]['roc_auc'])
y_pred_best=fitted[best].predict(X_test)
cm=confusion_matrix(y_test,y_pred_best)
sns.heatmap(cm,annot=True,fmt='d',cmap='Blues');plt.title(f'Confusion Matrix ({best})');plt.show()